# Deaths by Natural Disasters and Climate Change

*Sebastian Fürndraht, Hannes Rokitte, Paul Schmitt, Lukas Wieser*

## Overview
- Introduction
    - Research Questions
    - Used Datasets
    - Requirements & Dependencies
    - Constants
    - Download Temperature Data
    - Load Raw Datasets
- Data Integration
    - Preprocess CIA dataset
    - Determine ISO codes for temperature data
    - Which countries are in which datasets?
    - Which regions are in which datasets?
    - Which countries need to be manually assigned a region
- Prepare Datasets
    - ...
- Data Exploration
    - ...
- Conclusion
    - ...



## 1. Introduction

### 1.1. Research Questions

- How did the number of deaths per year from natural disasters change over the last hundred years?
- How does this vary by country?
- How does this vary by type of natural disaster?
- Are there trends visible that could be due to climate change?

### 1.2. Used Datasets

**Why we choose the datasets:**

We have 3 main datasets. For disaster data we have EM-DAT. For determining the deaths per population of a natural disaster we also have the Gapminder Population Dataset. And finally for analysing trends of natural disasters with climate change we have temperature data by Berkeley Earth Study.

Additionally, we have 2 auxiliary datasets for countries (UN Country Codes and CIA Country Codes). The temperature dataset only contains country names, and these datasets help us to derive the country codes. We need both of them since the CIA dataset is a bit outdated, and the UN dataset does only contain official UN countries, e.g. it excludes Taiwan. Additionally we also use the UN Dataset to derive the region of each country.

**List of the datasets:**

- EM-DAT:
    - content: Yearly deaths by disasters
    - timespan: 1900-2022
    - countries: 235
    - source: https://public.emdat.be/
    - license: this is the only dataset, which is not allowed to be published.
- Gapminder Population:
    - content: Yearly population. Globally, by country, and by region
    - timespan: 1800-2100
    - source: https://www.gapminder.org/data/documentation/gd003/
- Berkeley Earth Study:
    - content: Monthly temperature anomalies. Globally, by country, and by region
    - timespan: 1750-2020
    - source: https://berkeleyearth.org/data/
- UN Country Codes:
    - content: Country Names, ISO Codes, and Regions.
    - countries: 250
    - source: https://unstats.un.org/unsd/methodology/m49/
- CIA Country Codes:
    - content: Country Names and Country Codes
    - countries: 278
    - source: https://www.cia.gov/the-world-factbook/references/country-data-codes/


### 1.3. Requirements & Dependencies

This project was created using Python 3.9.7.
The exact versions of the dependencies can be installed with the following command.

In [1]:
#!pip install -r requirements.txt -q

In [2]:
import numpy as np
import pandas as pd
import urllib.parse
import requests
from pathlib import Path

### 1.4. Constants

In [3]:
# RAW
DIS_RAW_FILE = Path('data/raw/disaster/emdat_public_2022_12_22_full.xlsx')
TEMP_RAW_FOLDER = 'data/raw/temperature/'
PATH_COUNTRIES_LAND_FOLDER = 'countries-land/'
PATH_REGIONS_LAND_FOLDER = 'regions-land/'
TEMP_GLOBAL_LAND_OCEAN_FILE = 'global-land-ocean.txt'
TEMP_RAW_COUNTRIES_LIST_FILE = "data/raw/temperature/countries-list.csv"

UN_COUNTRY_CODES_FILE = 'data/raw/country-codes/un-country-codes.csv'
CIA_COUNTRY_CODES_FILE = "data/raw/country-codes/cia-country-codes.csv"

# Preprocessed (to be deleted)
DIS_PROCESSED_FOLDER = "data/processed/disaster"
DIS_PROCESSED_ALL_FILE = Path("data/processed/disaster/disaster-all.csv")

PATH_COUNTRIES_LIST_FILE = 'temp-countries-list.csv'
COUNTRIES_LIST_FILE = "temp-countries-list.csv"

TEMP_PROCESSED_FOLDER = "data/processed/temperature/"
TEMP_PROCESSED_COUNTRIES_LIST_FILE = "data/processed/temperature/temp-countries-list.csv"

POP_PROCESSED_FOLDER = "data/processed/population"
POP_PROCESSED_GLOBAL_FILE = 'data/processed/population/population-global.csv'
POP_PROCESSED_REGION_FILE = 'data/processed/population/population-region.csv'
POP_PROCESSED_COUNTRY_FILE = 'data/processed/population/population-country.csv'

### Create Directories

In [4]:
Path(DIS_PROCESSED_FOLDER).mkdir(parents=True, exist_ok=True)
Path(TEMP_PROCESSED_FOLDER).mkdir(parents=True, exist_ok=True)
Path(POP_PROCESSED_FOLDER).mkdir(parents=True, exist_ok=True)

### 1.5. Download Temperature Data
Automatically download the regional and country temperature data, so we don't have to download each file by ourselves.

In [5]:
countries = pd.read_csv(TEMP_RAW_COUNTRIES_LIST_FILE, sep=";")

In [6]:
temp_regions = countries["Region"].dropna().unique().tolist()
temp_countries = countries["Country"].tolist()

In [7]:
def download_temperature_countries(country_names: list[str]):
    for country in country_names:
        print(f"downloading {country}")
        country_encoded = urllib.parse.quote(country.lower().replace(" ", "-"),encoding='cp1252')
        url = f"http://berkeleyearth.lbl.gov/auto/Regional/TAVG/Text/{country_encoded}-TAVG-Trend.txt"
        response = requests.get(url)
        data = response.text
        with open(f'data/raw/temperature/countries-land/{country}.txt', 'w', encoding="utf-8") as file:
            file.write(data)

In [8]:
def download_temperature_regions(region_names: list[str]):
    for region in region_names:
        print(f"downloading {region}")
        region_encoded = urllib.parse.quote(region.lower().replace(" ", "-"),encoding='cp1252')
        url = f"http://berkeleyearth.lbl.gov/auto/Regional/TAVG/Text/{region_encoded}-TAVG-Trend.txt"
        response = requests.get(url)
        data = response.text
        with open(f'data/raw/temperature/regions-land/{region}.txt', 'w', encoding="utf-8") as file:
            file.write(data)

Change the variable `should_download_temperature_data` to `True`, to download the temperature data of countries & regions. (This should not be necessary, since the data should already be downloaded)

In [9]:
should_download_temperature_data = False
if should_download_temperature_data:
    download_temperature_countries(temp_countries)
    download_temperature_regions(temp_regions)

### 1.6. Load Raw Datasets

Next we load the raw datasets. With exception to the temperature dataset, since it is divided into multiple files (e.g. one file per country) and will be loaded later on.

In [10]:
dis = pd.read_excel(DIS_RAW_FILE, skiprows=6, sheet_name="emdat data")

temperature_countries = pd.read_csv("data/raw/temperature/countries-list.csv", sep=";")

pop_dict = pd.read_excel('data/raw/population/gapminder-population-v7.xlsx', sheet_name=['data-for-world-by-year', 'data-for-regions-by-year', 'data-for-countries-etc-by-year'])
pop_global_df = pop_dict.get('data-for-world-by-year')
pop_country_df = pop_dict.get('data-for-countries-etc-by-year')
pop_region_gapminder = pop_dict.get('data-for-regions-by-year')

un_country_codes = pd.read_csv(UN_COUNTRY_CODES_FILE, sep=";")
cia_country_codes = pd.read_csv(CIA_COUNTRY_CODES_FILE, sep="\t")

C:\Users\lukas\anaconda3\envs\dopp-a2\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## 2. Data Integration

In this section we want to ensure, that we can later combine our datasets without many problems.
For this we first add country codes to the temperature dataset, and then take a closer look which countries and regions occur in each dataset.

### 2.1 Preprocess CIA dataset

First do some preprocessing on the CIA dataset, since it is a bit messy as we can see below. For example different kind of country codes are stored in the same column.

In [11]:
cia_country_codes.head(3)

,Entity,GENC,ISO 3166,Stanag,Internet,Comment
0,Afghanistan,AFG,AF | AFG | 004,AFG,.af,-
1,Akrotiri,XQZ,- | - | -,-,-,-
2,Albania,ALB,AL | ALB | 008,ALB,.al,-


In [12]:
# split iso-codes in separate columns
cia_country_codes[["ISO-alpha2","ISO-alpha3","ISO-numeric"]] = cia_country_codes["ISO 3166"].str.split("|",2,expand=True)
cia_country_codes.drop(columns=["ISO 3166"], inplace=True)
# strip whitespaces from iso-codes
cia_country_codes[["ISO-alpha2","ISO-alpha3","ISO-numeric"]] = cia_country_codes[["ISO-alpha2","ISO-alpha3","ISO-numeric"]].apply(lambda x: x.str.strip())
# replace not existing iso-codes with NaN for more clarity
cia_country_codes["ISO-alpha2"].replace("-", np.nan, inplace=True)
cia_country_codes["ISO-alpha3"].replace("-", np.nan, inplace=True)
cia_country_codes["ISO-numeric"].replace("-", np.nan, inplace=True)
# show preprocessed cia data
cia_country_codes.head(3)

,Entity,GENC,Stanag,Internet,Comment,ISO-alpha2,ISO-alpha3,ISO-numeric
0,Afghanistan,AFG,AFG,.af,-,AF,AFG,004
1,Akrotiri,XQZ,-,-,-,NaN,NaN,NaN
2,Albania,ALB,ALB,.al,-,AL,ALB,008


### 2.2 Determine ISO codes for temperature data

Next we want to add ISO codes to the temperature dataset, since the temperature dataset only contains country names. This will help us later, when we want to relate countries in the disaster dataset to countries in the temperature dataset.

#### Remove aggregated countries.

We can see, that e.g. Denmark appears twice. This issue happens multiple times, and is due to the reason that some countries are aggregates of other countries e.g. `Denmark` consists of `Denmark (Europe)` also known as `Denmark Mainland`, and `Greenland`. The bearkley earth website has a worldmap on which the country is highlighted, this helped us to better understand what each of the conflicting countries is.

In [13]:
temperature_countries.iloc[55: 55+5]

,Country,Region
55,Cyprus,Asia
56,Czech Republic,Europe
57,Denmark,North America
58,Denmark (Europe),Europe
59,Djibouti,Africa


We decided to remove the "aggregated" country. Here is a list of the aggregate countries we removed, their individual parts still exists in the dataset:
- Denmark (Denmark Mainland, Greenland)
- France (France Mainland, French Guiana, French Polynesia, French Southern and Antarctic Lands)
- Netherlands (Netherlands Mainland, Sint Maarten, Curaçao, Aruba)
- United Kingdom (United Kingdom + Oversea territories such as Montserrat, Bermuda)

In [14]:
temperature_countries_remove = pd.DataFrame({
    "Country": ["Denmark","France", "Netherlands", "United Kingdom"],
    "Region": ["North America", np.nan, "Europe", "Europe"]
})
temperature_countries_cleaned = pd.concat([temperature_countries, temperature_countries_remove]).drop_duplicates(keep=False)

#### Rename Countries & Match ISO Codes

Some countries need to be renamed so that we find the matching country-code later. After renaming the countries we can join the country codes to the temperature data.

In [15]:
new_country_names = {
    "Denmark (Europe)": "Denmark",
    "France (Europe)": "France",
    "Netherlands (Europe)": "Netherlands",
    "United Kingdom (Europe)": "United Kingdom",
    "Åland": "Åland Islands",
    "Czech Republic": "Czechia",
    "Turkey": "Türkiye",
    "Svalbard and Jan Mayen": "Svalbard and Jan Mayen Islands",
    "Cape Verde": "Cabo Verde",
    "Turks and Caicas Islands": "Turks and Caicos Islands",
    "Swaziland": "Eswatini",
    "Macedonia": "North Macedonia",
    "Côte d'Ivoire": "Côte d’Ivoire",
    "Federated States of Micronesia": "Micronesia (Federated States of)",
    "South Georgia and the South Sandwich Isla": "South Georgia and the South Sandwich Islands",
    "Bonaire, Saint Eustatius and Saba": "Bonaire, Sint Eustatius and Saba",
    "Congo (Democratic Republic of the)": "Democratic Republic of the Congo",
    "South Korea": "Korea, South",
    "North Korea": "Korea, North",
    "Palestina": "State of Palestine"
}

temperature_countries_cleaned = temperature_countries_cleaned.replace({"Country": new_country_names}, inplace=False)

# left-join cia-country-codes and un-country-codes
temperature_countries_with_iso = temperature_countries_cleaned.merge(cia_country_codes,how="left",left_on='Country', right_on='Entity')[["Country","ISO-alpha3"]]
temperature_countries_with_iso = temperature_countries_with_iso.merge(un_country_codes,how="left",left_on='Country', right_on='Country or Area')[["Country","ISO-alpha3", "ISO-alpha3 Code"]]

# fill missing cia-country codes with un-country-codes
temperature_countries_with_iso["ISO-alpha3"].fillna(temperature_countries_with_iso["ISO-alpha3 Code"], inplace=True)
temperature_countries_with_iso.drop(columns=["ISO-alpha3 Code"], inplace=True)

# show countries for which we could not find an ISO code
temperature_countries_with_iso[temperature_countries_with_iso["ISO-alpha3"].isna()]

,Country,ISO-alpha3
18,Baker Island,NaN
113,Kingman Reef,NaN
161,Palmyra Atoll,NaN


These 3 countries/areas above do not have any country codes in general, and are quite small, so we just ignore them later on.

### 2.3 Which countries are in which datasets?

Next we take a look, if the datasets all have the same countries, or if there are countries that only exist in one dataset.

#### Disaster vs. Temperature Dataset

In [16]:
berkely_iso_codes = set(temperature_countries_with_iso["ISO-alpha3"].dropna().tolist())
emdat_iso_codes = set(dis["ISO"].unique().tolist())

emdat_and_bekely = emdat_iso_codes.intersection(berkely_iso_codes)
emdat_without_berkely = emdat_iso_codes-emdat_and_bekely
berkely_without_emdat = berkely_iso_codes-emdat_and_bekely

print(f"countries in disaster & temperature dataset: {len(emdat_and_bekely)}")
print(f"countries in disaster but not temperature dataset ({len(emdat_without_berkely)}):")
print(sorted(emdat_without_berkely))
print(f"countries in temperature but not disaster dataset ({len(berkely_without_emdat)}):")
print(sorted(berkely_without_emdat))

countries in disaster & temperature dataset: 209
countries in disaster but not temperature dataset (22):
['ANT', 'AZO', 'BMU', 'BRN', 'COK', 'CSK', 'DDR', 'DFR', 'MDV', 'MHL', 'SCG', 'SHN', 'SPI', 'SSD', 'SUN', 'TKL', 'TUV', 'VUT', 'WLF', 'YMD', 'YMN', 'YUG']
countries in temperature but not disaster dataset (20):
['ABW', 'ALA', 'AND', 'ATA', 'ATF', 'BES', 'CXR', 'ESH', 'FLK', 'FRO', 'GGY', 'GRL', 'HMD', 'JEY', 'LIE', 'MCO', 'SGS', 'SJM', 'SMR', 'SPM']


The composition of countries for which we have disaster data, but no temperature data is as follows:
- Existing Countries (usually very small countries/islands):
     - `AZO` Azores Islands, `BMU` Bermuda, `BRN` Brunei Darussalam, `COK` Cook Islands (the), `MDV` Maldives, `MHL` Marshall Islands (the), `SHN` Saint Helena, Ascension and Tristan da Cunha, `SSD` South Sudan, `TKL` Tokelau, `TUV` Tuvalu, `VUT` Vanuatu, `WLF` Wallis and Futuna
- Existing Countries (but invalid country code):
    - `SPI` Canary Islands
- Former Countries:
    - `ANT` Netherlands Antilles,`CSK` Czechoslovakia,`DDR` Germany Dem Rep,`DFR` Germany Fed Rep,`SCG` Serbia Montenegro,`SUN` Soviet Union,`YMD` Yemen P Dem Rep,`YMN` Yemen Arab Rep,`YUG` Yugoslavia

#### Disaster vs Population Dataset

In [17]:
gapminder_iso_codes = set(pop_country_df["geo"].str.upper().unique())

emdat_and_gapminder = emdat_iso_codes.intersection(gapminder_iso_codes)
emdat_without_gapminder = emdat_iso_codes-emdat_and_gapminder
gapminder_without_emdat = gapminder_iso_codes-emdat_and_gapminder

print(f"countries in disaster & population dataset: {len(emdat_and_gapminder)}")
print(f"countries in disaster but not population dataset ({len(emdat_without_gapminder)}):")
print(sorted(emdat_without_gapminder))
print(f"countries in population but not disaster dataset ({len(gapminder_without_emdat)}):")
print(sorted(gapminder_without_emdat))

countries in disaster & population dataset: 191
countries in disaster but not population dataset (40):
['AIA', 'ANT', 'ASM', 'AZO', 'BLM', 'BMU', 'COK', 'CSK', 'CUW', 'CYM', 'DDR', 'DFR', 'GLP', 'GUF', 'GUM', 'IMN', 'MAC', 'MAF', 'MNP', 'MSR', 'MTQ', 'MYT', 'NCL', 'NIU', 'PRI', 'PYF', 'REU', 'SCG', 'SHN', 'SPI', 'SUN', 'SXM', 'TCA', 'TKL', 'VGB', 'VIR', 'WLF', 'YMD', 'YMN', 'YUG']
countries in population but not disaster dataset (6):
['AND', 'HOS', 'LIE', 'MCO', 'NRU', 'SMR']


The Countries for which we have disaster data, but no population data are as follows:
- Existing Countries (independent)
    - `COK` Cook Islands (the), `NIU` Niue
- Existing Countries (dependent e.g .oversea territories)
	- `AIA` Anguilla, `ASM` American Samoa, `AZO` Azores Islands, `BLM` Saint Barthélemy, `BMU` Bermuda, `CUW` Curaçao, `CYM` Cayman Islands (the), `GLP` Guadeloupe, `GUF` French Guiana, `GUM` Guam, `IMN` Isle of Man, `MAC` Macao, `MAF` Saint Martin (French Part), `MNP` Northern Mariana Islands (the), `MSR` Montserrat, `MTQ` Martinique, `MYT` Mayotte, `NCL` New Caledonia, `PRI` Puerto Rico, `PYF` French Polynesia, `REU` Réunion, `SHN` Saint Helena, Ascension and Tristan da Cunha, `SPI` Canary Islands, `SXM` Sint Maarten (Dutch part), `TCA` Turks and Caicos Islands (the), `TKL` Tokelau, `VGB` Virgin Island (British), `VIR` Virgin Island (U.S.), `WLF` Wallis and Futuna
- Former Countries
	- `ANT` Netherlands Antilles, `CSK` Czechoslovakia, `DDR` Germany Dem Rep, `DFR` Germany Fed Rep, `SCG` Serbia Montenegro, `SUN` Soviet Union, `YMD` Yemen P Dem Rep, `YMN` Yemen Arab Rep, `YUG` Yugoslavia

#### Conclusion of comparing countries

When comparing data on the country level, we should keep in mind, that (1) countries can change over time, and (2) that for some countries should only occur in one dataset and thus should probably be ignored when trying to relate disasters with climate change.

We can later also handle former countries e.g. merge `DDR` and `DFR` into `DEU`.

### 2.4 Which regions are in which datasets?

The regions have different names for each dataset ("continent", "geo", "Region").

In [18]:
dis["Continent"].unique().tolist()

['Africa', 'Asia', 'Europe', 'Americas', 'Oceania']

In [19]:
pop_region_gapminder["geo"].unique().tolist()

['africa', 'asia', 'europe', 'americas']

In [20]:
temperature_countries["Region"].dropna().unique().tolist()

['Asia', 'Europe', 'Africa', 'South America', 'Oceania', 'North America']

As we can see, each dataset has a different number of regions. Additionally, we don't know which countries belong to each region. For example the region Europe could consist of different countries the disaster dataset than in the temperature dataset.

That's why we decided to **compute the regional data ourselves**, by aggregating the countries according to UN Regions.

### 2.5 Which countries need to be manually assigned a region

Check which countries are not in the UN dataset, and thus need to be manually assigned a region.

In [21]:
un_iso_codes = set(un_country_codes["ISO-alpha3 Code"].tolist())

emdat_without_un = emdat_iso_codes-un_iso_codes
gapminder_without_un = gapminder_iso_codes-un_iso_codes
berkely_without_un = berkely_iso_codes-un_iso_codes

print(f"countries in disaster, but not UN dataset ({len(emdat_without_un)}): \n{sorted(emdat_without_un)}")
print(f"countries in population, but not UN dataset ({len(gapminder_without_un)}): \n{sorted(gapminder_without_un)}")
print(f"countries in temperature, but not UN dataset ({len(berkely_without_un)}): \n{sorted(berkely_without_un)}")

countries in disaster, but not UN dataset (12): 
['ANT', 'AZO', 'CSK', 'DDR', 'DFR', 'SCG', 'SPI', 'SUN', 'TWN', 'YMD', 'YMN', 'YUG']
countries in population, but not UN dataset (2): 
['HOS', 'TWN']
countries in temperature, but not UN dataset (1): 
['TWN']


## 3. Prepare Datasets

In this section we apply preprocessing to our 3 datasets. This includes standardizing the column names, removing unnecessary columns, removing rows, handling missing values, deriving the region of each country, aggregating data, etc.

### 3.1 Disaster Data

The goal is to preprocess the disaster data for later use.

**What is considered as a disaster?**

The publisher of the dataset *Centre for Research on the Epidemiology of Disasters* (CRED) defines a disaster as “a situation or event that overwhelms local capacity, necessitating a request at the national or international level for external assistance; an unforeseen and often sudden event that causes great damage, destruction and human suffering”

For a disaster to be entered into the database at least one of the following criteria must be fulfilled:

- 10 or more people reported killed
- 100 or more people reported affected
- Declaration of a state of emergency
- Call for international assistance

First look at the disaster data:

In [22]:
dis.head()

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1900-9002-CPV,1900,9002,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
1,1900-9001-IND,1900,9001,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
2,1901-0003-BEL,1901,3,NaN,Technological,Technological,Industrial accident,Explosion,NaN,Coal mine,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
3,1902-0012-GTM,1902,12,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,25000.0,781207.0,3.200175,NaN,NaN,NaN,NaN
4,1902-0003-GTM,1902,3,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,...,NaN,NaN,NaN,NaN,NaN,3.200175,NaN,NaN,NaN,NaN


We can see that each row contains a single disaster, and corresponding information.

#### Select & Rename Attributes

Replace whitespaces with underscores, convert every character to lowercase, rename specific columns to ensure uniformity
and finally select only the interesting columns. Other columns like "economic" impact are removed, since they are out of scope for this task.

In [23]:
# Remove whitespaces from all col-names and convert them to lower-case
dis.columns = [c.replace(' ', '_').lower() for c in dis.columns]
dis.rename(columns={'country':'country_name', 'iso':'country_code', 'disaster_group':'group','disaster_subgroup':'subgroup', 'disaster_subtype':'subtype', 'disaster_type':'type', 'total_deaths':'deaths'}, inplace=True)

In [24]:
# Select the most interesting columns
dis_all_col_names = ["year", "dis_no", "country_name", "country_code", "location","group","subgroup", "type", "subtype", "deaths", "dis_mag_value", "dis_mag_scale", "start_year", "end_year"]
dis_all = dis.filter(items=dis_all_col_names)

#### Which disaster groups are present in the dataset ?

There are Natural disasters, technological disasters as well as complex disasters that represent specific events (e.g. famine) which are not directly linked to a natural hazard.

In [25]:
dis_all["group"].unique().tolist()

['Natural', 'Technological', 'Complex Disasters']

We only focus on disasters which have a natural causation

In [26]:
dis_all = dis_all[dis_all["group"] == "Natural"].copy()
dis_all.drop(columns="group", inplace=True)

Which types of natural disasters are there ?

In [27]:
dis_all.groupby(["subgroup","type"]).agg({"deaths":"sum"}).reset_index()

,subgroup,type,deaths
0,Biological,Animal accident,12.0
1,Biological,Epidemic,9618804.0
2,Biological,Insect infestation,0.0
3,Climatological,Drought,11733889.0
4,Climatological,Glacial lake outburst,262.0
5,Climatological,Wildfire,4653.0
6,Extra-terrestrial,Impact,0.0
7,Geophysical,Earthquake,2343912.0
8,Geophysical,Mass movement (dry),4644.0
9,Geophysical,Volcanic activity,86893.0


The types of disasters are mostly the ones a normal person would expect when thinking about natural disasters. But there are some strange types like `Insect Infestations` or `Animal Accident` which are not that obvious to understand, they also have basically no deaths. Also, for our research we want to exclude `Epidemics` since it would go beyond the scope of this task.

Therefore also decided to omit disasters of the subgroups `Biological` and `Extra-terrestrial`.

In [28]:
dis_all = dis_all[(dis_all["subgroup"] != "Biological") & (dis_all["subgroup"] != "Extra-terrestrial")]

We only consider the following types of disasters:

In [29]:
sorted(dis_all["type"].unique())

['Drought',
 'Earthquake',
 'Extreme temperature ',
 'Flood',
 'Fog',
 'Glacial lake outburst',
 'Landslide',
 'Mass movement (dry)',
 'Storm',
 'Volcanic activity',
 'Wildfire']

#### Handle Missing Values

Fill missing Values for the number of deaths

We can assume that missing values for the number of deaths of a particular disaster means that the deathtoll was 0.

For the subtype we take a look for which type of natural disasters a subtype is not provided.

In [30]:
dis_all.isna().sum()

year                0
dis_no              0
country_name        0
country_code        0
location         1449
subgroup            0
type                0
subtype          3116
deaths           4412
dis_mag_value    9903
dis_mag_scale    1114
start_year          0
end_year            0
dtype: int64

We can see that all disasters have a year, country, and type. However, especially for subtype and deaths it would be interesting to take a closer look, since the "type" of disasters and the "deaths" are integral parts of our analysis.

In [31]:
sorted(dis_all[dis_all["subtype"].isna()]["type"].unique())

['Drought',
 'Earthquake',
 'Flood',
 'Fog',
 'Glacial lake outburst',
 'Landslide',
 'Mass movement (dry)',
 'Storm',
 'Volcanic activity',
 'Wildfire']

Unfortunately the missing values in the subtype column do not correspond to specific types of disasters, but basically all diaster types have missing subtypes. We can not conclude that easily what caused the values to be missing.

In [32]:
dis_all[dis_all["type"] == "Drought"].subtype.unique()

array(['Drought', nan], dtype=object)

Disasters of type drought have no further more specific subtype. Consequently, we assign all droughts with missing values for subtype, the general type

In [33]:
dis_all.loc[dis_all["type"] == "Drought", "subtype"] = dis_all.loc[dis_all["type"] == "Drought", "subtype"].fillna("Drought")

For all other types, we unfortunately can not guess the subtype. Therefore, we assign a custom label "Uncategorized" + the type of the disaster for each missing subtype. We do not need to do this for disasters of type extreme temperature, since subtypes for all observations are specified

In [34]:
dis_all.loc[dis_all["type"] == "Earthquake", "subtype"] = dis_all.loc[dis_all["type"] == "Earthquake", "subtype"].fillna("Uncategorized_Earthquake")
dis_all.loc[dis_all["type"] == "Storm", "subtype"] = dis_all.loc[dis_all["type"] == "Storm", "subtype"].fillna("Uncategorized_Storm")
dis_all.loc[dis_all["type"] == "Flood", "subtype"] = dis_all.loc[dis_all["type"] == "Flood", "subtype"].fillna("Uncategorized_Flood")

We assume that disasters with no death toll reported have a death toll of 0. This also aligns with the information we get from the source of our dataset (deaths tolls smaller than 10 are not registered)

In [35]:
dis_all[['deaths']] = dis_all[['deaths']].fillna(value=0)

#### Determine Regions

As mentioned in `data integration` we want to compute the region of each disaster, by taking the UN Region that is assigned to each Country, in which the disaster occurred. Some country codes are not in the list of UN countries, thus we handle them specifically later.

Once more, here are the countries that need to be handled specifically:

In [36]:
dis_iso_codes = set(dis_all["country_code"].unique())
un_iso_codes = set(un_country_codes["ISO-alpha3 Code"].tolist())
emdat_without_un = dis_iso_codes-un_iso_codes
print(f"countries in disaster, but not un dataset ({len(emdat_without_un)}): \n{sorted(emdat_without_un)}")

countries in disaster, but not un dataset (12): 
['ANT', 'AZO', 'CSK', 'DDR', 'DFR', 'SCG', 'SPI', 'SUN', 'TWN', 'YMD', 'YMN', 'YUG']


**Automatically assign regions with UN Dataset**

For countries that were split in the past, but are now unified, we can just assign the unified country.

In [37]:
# Germany
dis_all.loc[dis_all['country_code'] == "DFR",'country_code'] = "DEU"
dis_all.loc[dis_all['country_code'] == "DDR",'country_code'] = "DEU"
# Yemen
dis_all.loc[dis_all['country_code'] == "YMD",'country_code'] = "YEM"
dis_all.loc[dis_all['country_code'] == "YMN",'country_code'] = "YEM"

Next we derive the region by using the UN dataset.

In [38]:
dis_all = pd.merge(dis_all, un_country_codes[["ISO-alpha3 Code","Region Name", "Region Code"]], left_on='country_code', right_on='ISO-alpha3 Code', how='left')
dis_all.rename(columns={"Region Code": "region_code", "Region Name": "region_name"}, inplace=True)
dis_all.drop(columns=["ISO-alpha3 Code"],inplace=True)

**Manually Assign Regions**

Some countries clearly belong to one region, so we can assign the disasters manually

In [39]:
# Taiwan
dis_all.loc[dis_all['country_code']=='TWN','region_name'] = 'Asia'
dis_all.loc[dis_all['country_code']=='TWN','region_code'] = 142
# Czechoslovakia
dis_all.loc[dis_all['country_code']=='CSK','region_name'] = 'Europe'
dis_all.loc[dis_all['country_code']=='CSK','region_code'] = 150
# Yugoslavia
dis_all.loc[dis_all['country_code']=='YUG','region_name'] = 'Europe'
dis_all.loc[dis_all['country_code']=='YUG','region_code'] = 150
# Serbia Montenegro
dis_all.loc[dis_all['country_code']=='SCG','region_name'] = 'Europe'
dis_all.loc[dis_all['country_code']=='SCG','region_code'] = 150
# Netherlands Antilles
dis_all.loc[dis_all['country_code']=='ANT','region_name'] = 'Americas'
dis_all.loc[dis_all['country_code']=='ANT','region_code'] = 19
# Azores Islands
dis_all.loc[dis_all['country_code']=='AZO','region_name'] = 'Europe'
dis_all.loc[dis_all['country_code']=='AZO','region_code'] = 150
# Canary Islands
dis_all.loc[dis_all['country_code']=='SPI','region_name'] = 'Europe'
dis_all.loc[dis_all['country_code']=='SPI','region_code'] = 150

**Manually Assign Regions of Soviet Union**

For the Soviet Union disasters can occur in the european and/or asian parts. Thus we also take the "location" attribute into account and try to derive if the disaster was in europe or asia.

In [40]:
mask_europe = dis_all.loc[dis_all['country_code']=='SUN']["location"].str.contains("Russian Federation|Ukraine|Moldavia|Siberia").fillna(False)
mask_asia = dis_all.loc[dis_all['country_code']=='SUN']["location"].str.contains("Kazakhstan|Azerbaijan|Uzbekistan|Turkmenistan|Georgia|Armenia|Kyrgystan|Tajikistan|Tajiskistan|Tadzhikistan|Tadjikistan|Caucasus region|Dushanbe", case=False).fillna(False)

In [41]:
dis_all[dis_all['country_code']=='SUN'][mask_europe & mask_asia]

,year,dis_no,country_name,country_code,location,subgroup,type,subtype,deaths,dis_mag_value,dis_mag_scale,start_year,end_year,region_name,region_code
849,1921,1921-9001-SUN,Soviet Union,SUN,"South Ukraine, Volga, Ural (Kazakhstan,Russian...",Climatological,Drought,Drought,1200000.0,NaN,Km2,1921,1921,NaN,NaN


Only one Event happened in both the asian and the european part of the soviet union. It is also a major event since it is a drought which caused the death of 1.2 million people.

Researching the details of this event one can conclude that this observation can only be the [Russian famine of 1921–1922](https://en.wikipedia.org/wiki/Russian_famine_of_1921%E2%80%931922). It mostly affected people living in europe, hence we assign this single observation the region europe.

For all other observations, the region should be unambiguous.

In [42]:
dis_all.loc[(dis_all['country_code']=='SUN') & mask_europe, "region_name"] = "Europe"
dis_all.loc[(dis_all['country_code']=='SUN') & mask_europe, "region_code"] = 150

dis_all.loc[(dis_all['country_code']=='SUN') & mask_asia, "region_name"] = "Asia"
dis_all.loc[(dis_all['country_code']=='SUN') & mask_asia, "region_code"] = 142

dis_all.loc[dis_all['dis_no']=='1921-9001-SUN', "region_name"] = "Europe"
dis_all.loc[dis_all['dis_no']=='1921-9001-SUN', "region_code"] = 150

The only disasters without a region are now these 3 in the soviet Union. However, since they have no death count and are just 3 disaster, we can safely ignore them.

In [43]:
dis_all[dis_all["region_code"].isna()]

,year,dis_no,country_name,country_code,location,subgroup,type,subtype,deaths,dis_mag_value,dis_mag_scale,start_year,end_year,region_name,region_code
3187,1981,1981-0280-SUN,Soviet Union,SUN,East,Meteorological,Storm,Uncategorized_Storm,0.0,NaN,Kph,1981,1981,NaN,NaN
3188,1981,1981-0301-SUN,Soviet Union,SUN,NaN,Hydrological,Flood,Uncategorized_Flood,0.0,NaN,Km2,1981,1981,NaN,NaN
3213,1982,1982-0346-SUN,Soviet Union,SUN,East,Hydrological,Flood,Uncategorized_Flood,0.0,150.0,Km2,1982,1982,NaN,NaN


#### Look at the preprocessed data

The preprocessed disaster data now looks like this:

In [44]:
# Todo: Remove later
dis_all.to_csv(DIS_PROCESSED_ALL_FILE, index=False)

In [45]:
dis_all.head(3)

,year,dis_no,country_name,country_code,location,subgroup,type,subtype,deaths,dis_mag_value,dis_mag_scale,start_year,end_year,region_name,region_code
0,1900,1900-9002-CPV,Cabo Verde,CPV,Countrywide,Climatological,Drought,Drought,11000.0,NaN,Km2,1900,1900,Africa,2.0
1,1900,1900-9001-IND,India,IND,Bengal,Climatological,Drought,Drought,1250000.0,NaN,Km2,1900,1900,Asia,142.0
2,1902,1902-0012-GTM,Guatemala,GTM,"Quezaltenango, San Marcos",Geophysical,Earthquake,Ground movement,2000.0,8.0,Richter,1902,1902,Americas,19.0


### 3.2 Population Data

The goal is to preprocess the population dataset. Specifically we want to create 3 datasets, one for the global level, one for the regional level, and one for the country level.

We consider the period from 1900 until 2021 and remove redundant rows. We do this because the data after 2021 are based on estimated values, and are basically predictions into the future.

Since the regions of the dataset did not match with the regions of our other datasets, we chose to compute the regions-population according to the regions from the un-country-codes.csv file.

**How exactly is the population determined?**
The primary data from the sources originates from census, informal census, indirect estimate and arbitrary guesses.
For the years 1800-1950 the underlying data comes from the `Angus Maddison` dataset, and from 1950 onwards the data is provided by the World Population Prospects (WPP) by the UN. Therefore, we suspect that the data from 1950 onwards is more reliable, since it was collected in a more standardized manner by the UN.


#### Check for missing values

In [46]:
pop_global_df.isna().sum()

geo           0
name          0
time          0
Population    0
dtype: int64

In [47]:
pop_country_df.isna().sum()

geo             0
name            0
time            0
Population    100
dtype: int64

In [48]:
pop_country_df[pop_country_df["Population"].isna()]["geo"].unique()

array(['hos'], dtype=object)

We see that only `Holy Sea` has missing values, we will keep this in mind, and come back later.

Does the regional data have missing values?

In [49]:
pop_region_gapminder.isna().sum()

geo           0
name          0
time          0
Population    0
dtype: int64

#### Preprocess global data

In [50]:
pop_global_df.head(3)

,geo,name,time,Population
0,world,world,1800.0,985083734.9
1,world,world,1801.0,988518009.0
2,world,world,1802.0,991993182.0


Remove & rename columns

In [51]:
pop_global_df = pop_global_df[['time', 'Population']]
pop_global_df.rename(columns={'Population': 'population', 'time': 'year'}, inplace=True)
pop_global_df.set_index('year')

C:\Users\lukas\anaconda3\envs\dopp-a2\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,population
year,
1800.0,9.850837e+08
1801.0,9.885180e+08
1802.0,9.919932e+08
1803.0,9.955096e+08
1804.0,9.990675e+08
...,...
2096.0,1.038852e+10
2097.0,1.037989e+10
2098.0,1.037045e+10


Extract population data from 1900 until 2021

In [52]:
pop_global_df = pop_global_df[pop_global_df['year'] >= 1900]
pop_global_df = pop_global_df[pop_global_df['year'] <= 2021]
pop_global_df['population'] = pop_global_df['population'].astype(int)

In [53]:
pop_global_df.head()

,year,population
100,1900.0,1627123965
101,1901.0,1639684222
102,1902.0,1652898195
103,1903.0,1666760587
104,1904.0,1680799825


#### Prepare dataset for countries and regions

Extract population data from 1900 until 2021

In [54]:
pop_country_df = pop_country_df[pop_country_df['time'] >= 1900]
pop_country_df = pop_country_df[pop_country_df['time'] <= 2021]

Derive the region of each country by using the UN dataset.

In [55]:
un_country_codes = un_country_codes[['Region Code', 'Region Name', 'ISO-alpha3 Code']]

# format column that will be joined
pop_country_df['geo'] = pop_country_df['geo'].str.upper()

# merge country-population and un-country-codes
pop_country_region_df = pd.merge(pop_country_df, un_country_codes, how='left', left_on='geo', right_on='ISO-alpha3 Code')
pop_country_region_df.rename(columns={'Region Code': 'region_code', 'Region Name': 'region_name', 'time': 'year', 'Population': 'population', 'geo': 'country_code', 'name': 'country_name'}, inplace=True)

# remove unused column 'ISO-alpha3 Code'
pop_country_region_df.drop('ISO-alpha3 Code', axis=1, inplace=True)

Manually add the region to the countries which are not included in the UN dataset

In [56]:
# holy see
pop_country_region_df.loc[pop_country_region_df['country_code'] == 'HOS', 'region_code'] = 150
pop_country_region_df.loc[pop_country_region_df['country_code'] == 'HOS', 'region_name'] = 'Europe'
# taiwan
pop_country_region_df.loc[pop_country_region_df['country_code'] == 'TWN', 'region_code'] = 142
pop_country_region_df.loc[pop_country_region_df['country_code'] == 'TWN', 'region_name'] = 'Asia'

Check if we worked correctly

In [57]:
pop_country_region_df["country_code"].isna().sum()

0

However `holy see` still has missing values as we saw earlier.

In [58]:
pop_country_region_df[pop_country_region_df["country_code"]=="HOS"]["population"].max()

911.0

The remaining missing values are population data for the country 'holy see' since the missing population is at maximum smaller than 1000 decided to exclude holy see from our analysis. Having no missing values helps us later, since can convert the datatype of the columns from float to integers.

In [59]:
pop_country_region_df = pop_country_region_df[pop_country_region_df["country_code"] != "HOS"]

#### Adjust Datatypes

In [60]:
pop_country_region_df["population"] = pop_country_region_df["population"].astype(np.int64)
pop_country_region_df["year"] = pop_country_region_df["year"].astype(int)

pop_global_df["year"] = pop_global_df["year"].astype(int)

pop_region_gapminder["Population"] = pop_region_gapminder["Population"].astype(np.int64)
pop_region_gapminder["time"] = pop_region_gapminder["time"].astype(int)

#### Derive regional & country data

In [61]:
columns_regions = ['region_code', 'region_name', 'country_code', 'country_name', 'year', 'population']
pop_regions_df = pop_country_region_df[columns_regions]
pop_regions_df = pop_regions_df.groupby(['region_code', 'region_name', 'year'], as_index=False)['population'].sum()
#pop_regions_df['population'] = pop_regions_df['population'].astype(np.int64)
asia = pop_regions_df[pop_regions_df['region_name'] == 'Asia']

In [62]:
columns_country = ['country_code', 'country_name', 'year', 'population']
pop_country_df = pop_country_region_df[columns_country]

#### Compare our calculated regional data to the gapminder

To check if we aggregated our regional data correctly from the country data, we comapre it to the data from original regional data from gapminder.

In [63]:
regions_population_2021_gapminder = pop_region_gapminder[pop_region_gapminder['time'] == 2021]
total_population_gapminder = regions_population_2021_gapminder['Population'].sum()

regions_population_2021_calc = pop_regions_df[pop_regions_df['year'] == 2021]
total_population_calc = regions_population_2021_calc['population'].sum()

Total population from the year 2021

In [64]:
print(f'Gapminder:  {total_population_gapminder:.0f}')
print(f'Calculated: {total_population_calc:.0f}')

Gapminder:  7898737625
Calculated: 7898737625


Gapminder population by region for 2021:

In [65]:
regions_population_2021_gapminder[['name', 'Population']]

,name,Population
221,africa,1391823318
522,asia,4634610444
823,europe,846050284
1124,americas,1026253579


Our population by region for 2021:

In [66]:
regions_population_2021_calc[['region_name', 'population']]

,region_name,population
121,Africa,1391823318
243,Oceania,43602426
365,Americas,1026253579
487,Asia,4693889556
609,Europe,743168746


As we can see, we get the same results for the total populations in the year 2021.

If we take a look at the region population, we see that the calculated population of america and africa shows no difference to the gapminder-dataset population. For the regions asia and europe, we get different results. Furthermore, the calculated region oceania does not exist in the gapminder dataset. Since the documentation of the gapminder dataset does not include the country-to-region assignment, we cannot compare the differences between our calculation and gapminder. However, since the overall score is equal, we can argue that those differences occur tue to different country-to-region assignments.

In [67]:
# Todo: Remove later
pop_global_df.to_csv(POP_PROCESSED_GLOBAL_FILE, sep=';', index=False, header=True)
#store in csv file
pop_country_df.to_csv(POP_PROCESSED_COUNTRY_FILE, sep=';', index=False, header=True)
#store in csv file
pop_regions_df.to_csv(POP_PROCESSED_REGION_FILE, sep=';', index=False, header=True)

#### Look at the preprocessed data

In [68]:
pop_global_df.head(3)

,year,population
100,1900,1627123965
101,1901,1639684222
102,1902,1652898195


In [69]:
pop_country_df.head(3)

,country_code,country_name,year,population
0,AFG,Afghanistan,1900,4707744
1,AFG,Afghanistan,1901,4751177
2,AFG,Afghanistan,1902,4802500


In [70]:
pop_regions_df.head(3)

,region_code,region_name,year,population
0,2.0,Africa,1900,138578556
1,2.0,Africa,1901,139018147
2,2.0,Africa,1902,139489077


### 3.3 Temperature Data

The goal is to preprocess the temperature dataset. Specifically we want to create 3 datasets, one for the global level, one for the regional level, and one for the country level.

Since the regions of the dataset did not match with the regions of our other datasets, we chose to derive the regions ourselves according to the UN dataset.

We also need to do combine multiple files, since the data for each country and each region is located in a desperate file.

Additionally, we aggregate the data by the year, since the other datasets also operate on a yearly basis.

#### Load country level data

In this section we load Berkleys land-temperature data for each country from the corresponding .txt files and combine them into a single dataframe. We also change the naming of the countries according to the naming standard of the UN and add their country code.

In [71]:
temp_countries = temperature_countries_with_iso

In [72]:
# manually define the column names, since the raw data unfortunately no easy way to retrieve this information.
temp_land_country_column_names = ["country_code", "country_name", "year", "month", "monthly_anomaly", "monthly_unc", "annual_anomaly", "annual_unc", "five_year_anomaly", "five_year_unc", "ten_year_anomaly", "ten_year_unc", "twenty_year_anomaly", "twenty_year_unc"]
temp_country = pd.DataFrame(columns=temp_land_country_column_names)

temp_country_names = temp_countries['Country']

# Todo when combining use the dictionary in the data-integration notebook
new_country_names = {
    "Denmark (Europe)": "Denmark",
    "France (Europe)": "France",
    "Netherlands (Europe)": "Netherlands",
    "United Kingdom (Europe)": "United Kingdom",
    "Åland": "Åland Islands",
    "Czech Republic": "Czechia",
    "Turkey": "Türkiye",
    "Svalbard and Jan Mayen": "Svalbard and Jan Mayen Islands",
    "Cape Verde": "Cabo Verde",
    "Turks and Caicas Islands": "Turks and Caicos Islands",
    "Swaziland": "Eswatini",
    "Macedonia": "North Macedonia",
    "Côte d'Ivoire": "Côte d’Ivoire",
    "Federated States of Micronesia": "Micronesia (Federated States of)",
    "South Georgia and the South Sandwich Isla": "South Georgia and the South Sandwich Islands",
    "Bonaire, Saint Eustatius and Saba": "Bonaire, Sint Eustatius and Saba",
    "Congo (Democratic Republic of the)": "Democratic Republic of the Congo",
    "South Korea": "Korea, South",
    "North Korea": "Korea, North",
    "Palestina": "State of Palestine"
}

def swap_keys_values(d):
    return {v: k for k, v in d.items()}

map_country_to_filename = swap_keys_values(new_country_names)

for temp_country_name in temp_country_names:
    temp_country_file_name = temp_country_name
    if temp_country_name in map_country_to_filename:
        temp_country_file_name = map_country_to_filename[temp_country_name]
    path_temp_country_txt_file = TEMP_RAW_FOLDER + PATH_COUNTRIES_LAND_FOLDER + temp_country_file_name + '.txt'
    temp_land_one_country = pd.read_csv(path_temp_country_txt_file, comment="%", header=None, delim_whitespace=True)
    country_iso_name = temp_countries.loc[temp_countries['Country'] == temp_country_name]['ISO-alpha3'].iloc[0]

    temp_land_one_country.insert(0, 'country_code', country_iso_name)
    temp_land_one_country.insert(1, 'country_name', temp_country_name)

    temp_land_one_country.columns=temp_land_country_column_names

    temp_country = pd.concat([temp_country, temp_land_one_country])

temp_country.head()

,country_code,country_name,year,month,monthly_anomaly,monthly_unc,annual_anomaly,annual_unc,five_year_anomaly,five_year_unc,ten_year_anomaly,ten_year_unc,twenty_year_anomaly,twenty_year_unc
0,AFG,Afghanistan,1848,5,-0.297,2.037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1848,6,-0.796,2.136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1848,7,-0.113,1.937,-0.777,0.639,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1848,8,-0.462,1.937,-0.743,0.644,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1848,9,-1.272,1.865,-0.676,0.669,NaN,NaN,NaN,NaN,NaN,NaN


#### Derive regional from country temperature

**Why we are doing this:**
Berkleys offers a dataset with regional temperature anomaly data, however because we also use datasets for natural disasters and population we have to ensure that regions in all datasets are including the same countries.
As we saw in `data integration`, the regions defined for Berkleys regional temp anomaly datasets differ from the definition of regions in the UN dataset we use, therefore we decided to calculate regional temp anomalies based on the berkleys temp anomaly data for individual countries for regions based on the region data of the UN dataset.

**How to aggregate temperature anomalies:**
Temperature Anomalies tend to be similar across large regions geographical regions, even if the absolute temperature of two different measuring points differs for the same time period, their anomalies tend to be quite similar. To create regional temperature anomaly data we calculate the mean of temperature anomaly measures for all countries in a given region on a monthly basis. Source: https://data.giss.nasa.gov/gistemp/faq/#q101

When computing a region it is possible that some countries are not included, because it is possible that countries started reporting earlier than others or stopped reporting for some time periods.

In [73]:
def calc_region_anomaly(temp_land_country_regions):
    temp_return_land_region = temp_land_country_regions.groupby(['region_code', 'region_name', 'year', 'month'], as_index=False)['monthly_anomaly'].mean()
    temp_return_land_region.rename({ 'monthly_anomaly': 'temperature_anomaly'}, axis=1,inplace=True)
    return temp_return_land_region

In [74]:
# Todo: only load UN country codes once after combining notebook
# load and format un-country-codes data
un_country_codes = un_country_codes[['Region Code', 'Region Name', 'ISO-alpha3 Code']]
un_country_codes.columns=['region_code', 'region_name', 'country_code']

# Join UN country codes to temp data
# exclude temp data for antarctica when calculating regional temp data
temp_land_country_no_antarctica = temp_country[temp_country.country_name != 'Antarctica']
temp_land_country_regions = pd.merge(temp_land_country_no_antarctica, un_country_codes, on='country_code', how='left')
temp_land_country_regions.loc[temp_land_country_regions['country_name']=='Taiwan','region_name'] = 'Asia'
temp_land_country_regions.loc[temp_land_country_regions['country_name']=='Taiwan','region_code'] = 142

# calculate anomaly data for regions
temp_region = calc_region_anomaly(temp_land_country_regions)

temp_region.head()

,region_code,region_name,year,month,temperature_anomaly
0,2.0,Africa,1787,1,0.1520
1,2.0,Africa,1787,2,-0.3142
2,2.0,Africa,1787,3,-0.7876
3,2.0,Africa,1787,4,-0.9146
4,2.0,Africa,1787,5,-0.5570


#### Load berkleys regional data

We load Berkleys regional temperature anomaly data to compare it to the regional temperature anomaly data we calculated from the country temperature anomaly data and UN regions combined.

We combine Berkleys North and South America region to one Region 'Americas' to be able to compare it to the combined 'Americas' region of the UN data later.

In [75]:
# Todo: make this code segment prettier; reuse other code segments

temp_land_country_column_names = ["region_name", "year", "month", "monthly_anomaly", "monthly_unc", "annual_anomaly", "annual_unc", "five_year_anomaly", "five_year_unc", "ten_year_anomaly", "ten_year_unc", "twenty_year_anomaly", "twenty_year_unc"]
temp_berkley_regions = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']
temp_berkleys_region = pd.DataFrame(columns=temp_land_country_column_names)

for temp_berkley_region in temp_berkley_regions:
    region_file_name = temp_berkley_region
    path = TEMP_RAW_FOLDER + PATH_REGIONS_LAND_FOLDER + region_file_name + '.txt'
    one_region = pd.read_csv(path, comment="%", header=None, delim_whitespace=True)
    one_region.insert(0, 'region_name', temp_berkley_region)
    one_region.columns=temp_land_country_column_names
    temp_berkleys_region = pd.concat([temp_berkleys_region, one_region])

temp_berkleys_region.loc[(temp_berkleys_region['region_name'] == 'South America') | (temp_berkleys_region['region_name'] == 'North America'), 'region_name'] = 'Americas'
temp_berkleys_region = temp_berkleys_region[['region_name', 'year', 'month', 'monthly_anomaly']]
temp_berkleys_region = temp_berkleys_region.groupby(by=['region_name', 'year', 'month'])['monthly_anomaly'].mean()
temp_berkleys_region = temp_berkleys_region.reset_index()
temp_berkleys_region = temp_berkleys_region.rename(columns={'monthly_anomaly': 'temperature_anomaly'})

temp_berkleys_region.head(3)

,region_name,year,month,temperature_anomaly
0,Africa,1880,8,-0.181
1,Africa,1880,9,-0.389
2,Africa,1880,10,-0.274


#### Load global temperature data

Two versions exist that treat temperature anomalies at locations with sea ice:
1. Anomalies are extrapolated from land-surface air temperature anomalies.
2. Anomalies are extrapolated from sea-surface water temperature anomalies (usually collected from open water areas on the periphery of the sea ice).

We choose to use the air temperature version based on Berkleys remark:
"We believe that the use of air temperatures above sea ice provides a more natural means of describing changes in Earth's surface temperature."

In [76]:
# only include the dataset where anomalies are extrapolated from land-surface air temperature anomalies.
temp_global = pd.read_csv(TEMP_RAW_FOLDER + TEMP_GLOBAL_LAND_OCEAN_FILE, comment="%", header=None, delim_whitespace=True, engine='python', skipfooter=2079)
temp_global.columns = ["year", "month", "monthly_anomaly", "monthly_unc", "annual_anomaly", "annual_unc", "five_year_anomaly", "five_year_unc", "ten_year_anomaly", "ten_year_unc", "twenty_year_anomaly", "twenty_year_unc"]
temp_global.head()

,year,month,monthly_anomaly,monthly_unc,annual_anomaly,annual_unc,five_year_anomaly,five_year_unc,ten_year_anomaly,ten_year_unc,twenty_year_anomaly,twenty_year_unc
0,1850,1,-0.736,0.389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1850,2,-0.202,0.526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1850,3,-0.363,0.333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1850,4,-0.589,0.334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1850,5,-0.614,0.217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Remove unnecessary columns & rows

We only keep entries after the year 1900 because our research questions focus on the past 100 years.

In [77]:
def cut_before_1900(temp_data):
    return temp_data[temp_data['year'] >= 1900]

We only keep monthly anomaly data and rename the column to temperature_anomaly

In [78]:
def keep_monthly_anomalies(temp_data):
    drop_labels = ["monthly_unc", "annual_anomaly", "annual_unc", "five_year_anomaly", "five_year_unc", "ten_year_anomaly", "ten_year_unc", "twenty_year_anomaly", "twenty_year_unc"]
    temp_data = temp_data.drop(labels=drop_labels, axis=1)
    return temp_data.rename(columns={'monthly_anomaly': 'temperature_anomaly'})

Apply formatting functions for only keeping monthly anomalies and cut values before 1900.

In [79]:
# country temperature data
temp_country = cut_before_1900(temp_country)
temp_country = keep_monthly_anomalies(temp_country)
temp_country = temp_country.groupby(["year","country_code"]).agg({"temperature_anomaly":"mean","country_name":"first"})
temp_country.reset_index(inplace=True)
# regional temperature data (ours & berkely)
temp_region = cut_before_1900(temp_region)
temp_berkleys_region = cut_before_1900(temp_berkleys_region)
temp_region = temp_region.groupby(["year","region_code"]).agg({"temperature_anomaly":"mean","region_name":"first"})
temp_region.reset_index(inplace=True)
temp_berkleys_region.reset_index(inplace=True)
temp_berkleys_region = temp_berkleys_region.groupby(["year", "region_name"]).agg({"temperature_anomaly":"mean"})
temp_berkleys_region.reset_index(inplace=True)
# global temperature data
temp_global = cut_before_1900(temp_global)
temp_global = keep_monthly_anomalies(temp_global)
temp_global = temp_global.groupby(["year"])["temperature_anomaly"].mean()
temp_global = temp_global.reset_index()

#### Compare berkleys and our regional data

In [80]:
def temp_calc_region_corr(temp_single_region):
    temp_land_single_region = temp_region[(temp_region['region_name'] == temp_single_region)].set_index(['year'])
    temp_berkleys_land_single_region = temp_berkleys_region[(temp_berkleys_region['region_name'] == temp_single_region)].set_index(['year'])

    comp_temp_land_africa_joined = temp_berkleys_land_single_region.join(temp_land_single_region, on=['year'], lsuffix='_berkleys', rsuffix='_self')

    temp_land_region_corr = comp_temp_land_africa_joined['temperature_anomaly_berkleys'].corr(comp_temp_land_africa_joined['temperature_anomaly_self'])

    print(f'Correlation for region {temp_single_region:<10}: {temp_land_region_corr:0.2f}')

In [81]:
temp_berkley_regions = ['Africa', 'Asia', 'Europe', 'Americas', 'Oceania']
for temp_berkley_region in temp_berkley_regions:
    temp_calc_region_corr(temp_berkley_region)

Correlation for region Africa    : 0.99
Correlation for region Asia      : 0.87
Correlation for region Europe    : 0.99
Correlation for region Americas  : 0.94
Correlation for region Oceania   : 0.85


We can see that there are some sligth differences, which is expectable since the region differ, as we saw in `data integration`.

#### Handle Missing Values

In [82]:
print("Country: " + str(temp_country['temperature_anomaly'].isna().sum()))
print("Region:  " + str(temp_region['temperature_anomaly'].isna().sum()))
print("Global:  " + str(temp_global['temperature_anomaly'].isna().sum()))

Country: 5
Region:  0
Global:  0


Lets look in which countries the missing values occur

In [83]:
temp_country[temp_country['temperature_anomaly'].isna()].groupby('country_code').size()

country_code
GUM    2
PNG    2
SLB    1
dtype: int64

We do simple linear interpolation to for these 5 missing data values. The reson for this is, that we assume that the temperature for a given year is related to its previous and following year. Also since we only have 5 instances, there wont be a large effect on the data as a whole.

In [84]:
temp_country['temperature_anomaly'] = temp_country['temperature_anomaly'].interpolate(method='linear')

Check if we have removed all missing values

In [85]:
temp_country[temp_country['temperature_anomaly'].isna()==True].size

0

#### Look at the preprocessed data

In [86]:
# Todo: Remove later
temp_country.to_csv(TEMP_PROCESSED_FOLDER + 'temp-land-country.csv', index=False)
temp_region.to_csv(TEMP_PROCESSED_FOLDER + 'temp-land-region.csv', index=False)
temp_global.to_csv(TEMP_PROCESSED_FOLDER + 'temp-land-ocean-global.csv', index=False)

In [87]:
temp_country.head(3)

,year,country_code,temperature_anomaly,country_name
0,1900,ABW,-0.061750,Aruba
1,1900,AFG,-0.427250,Afghanistan
2,1900,AGO,0.146167,Angola


In [88]:
temp_region.head(3)

,year,region_code,temperature_anomaly,region_name
0,1900,2.0,-0.016705,Africa
1,1900,9.0,-0.307362,Oceania
2,1900,19.0,-0.160803,Americas


In [89]:
temp_global.head(3)

,year,temperature_anomaly
0,1900,-0.125167
1,1901,-0.199333
2,1902,-0.362167
